In [1]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD
!pwd

In [2]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Using <cuda> device


In [3]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [4]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 41

# ====== Dataset ====== #
args.img_type = 'lwir'
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)
args.ch_option = {'num_ch': 1,
                  'img_type': 'lwir',
                  'one_ch_option': 'mean'
                  }


# ====== Optimizer & Training ====== #
args.optim = 'Adam'
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 200
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [5]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-104',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-106
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## copy-paste augmentation

In [6]:
def limit_point_boundary(bbox, boundary=(0, 0, 300, 300)):
    for i in range(2):
        if bbox[i] < 0:
            bbox[i] = 0
    for i in range(2, 4):
        if bbox[i] > 300:
            bbox[i] = 300

    return bbox

def get_all_objects(images, bboxes):
    img_w, img_h = images.size(3), images.size(2)
    dims = torch.FloatTensor((img_w, img_h, img_w, img_h)).unsqueeze(0)
    bboxes = map(lambda x: torch.round(dims * x), bboxes)
    
    all_objects = []
    img_idx = []
    for i, (image, bboxes_in_img) in enumerate(zip(images, bboxes)):
        for bbox in bboxes_in_img:
            bbox = torch.round(bbox).long()
            x_min, y_min, x_max, y_max = limit_point_boundary(bbox)
            obj = image[:, y_min:y_max+1, x_min:x_max+1]
            img_idx.append(i)
            all_objects.append(obj)

    return all_objects, img_idx

In [7]:
def flip_objects(all_objects):
    for i in range(len(all_objects)):
        if random.random() < 0.5:
            all_objects[i] = torch.flip(all_objects[i], dims=(0, 2))

    return all_objects

In [8]:
def get_to_imgs(img_idxs, batch_size):
    counts = [0] * batch_size
    for i in img_idxs:
        counts[i] += 1
    max_count, min_count = max(counts), min(counts)
    weights = [max_count - count + min_count for count in counts]
    to_imgs = random.choices(
        range(batch_size), weights=weights, k=len(img_idxs))
    
    return to_imgs

In [9]:
def copy_paste_aug(images, bboxes, category_ids,
                   max_paste_try=5,
                   resize_min_max=(0.8, 2),
                   max_overlap=0
                   ):
    batch_size, num_ch, img_h, img_w = images.size()
    img_dim = torch.Tensor([img_w, img_h, img_w, img_h])
    all_objects, img_idxs = get_all_objects(images, bboxes)
    all_objects = flip_objects(all_objects)
    to_imgs = get_to_imgs(img_idxs, batch_size)

    for obj, to_img in zip(all_objects, to_imgs):
        _, obj_h, obj_w = obj.size()
        max_rescale = min(img_w / obj_w, img_h / obj_h)
        bboxes_in_img = bboxes[to_img]  # Tensor (n_objects, 4)
        cat_ids_in_img = category_ids[to_img]
        for _ in range(max_paste_try):
            rescale = random.uniform(*resize_min_max)
            rescale = min(rescale, max_rescale)
            r_h, r_w = int(rescale*obj_h), int(rescale*obj_w)
            r_h = 1 if r_h == 0 else r_h
            r_w = 1 if r_w == 0 else r_w
            resize_obj = F.interpolate(
                obj.unsqueeze(0), size=(r_h, r_w), mode='nearest'
            ).squeeze(0)
            paste_x = random.randint(0, img_w - r_w)
            paste_y = random.randint(0, img_h - r_h)
            paste_box = torch.Tensor(
                [paste_x, paste_y, paste_x+r_w, paste_y+r_h])   # (4,)
            scaled_paste_box = (paste_box / img_dim).unsqueeze(0)   # (1, 4)
            overlaps = find_jaccard_overlap(
                scaled_paste_box, bboxes_in_img
            )
            if overlaps.max().item() <= max_overlap:
                x_min, y_min, x_max, y_max = paste_box.long()
                images[to_img, :, y_min:y_max, x_min:x_max] = resize_obj
                bboxes_in_img = torch.cat((bboxes_in_img, scaled_paste_box))
                bboxes[to_img] = bboxes_in_img
                cat_ids_in_img = torch.cat((cat_ids_in_img,
                                           torch.LongTensor([1])))
                category_ids[to_img] = cat_ids_in_img
                break
            

    return images, bboxes, category_ids

## train

In [10]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds) in enumerate(train_loader):
        data_time.update(time.time() - start)
        images, bboxes, category_ids = \
            copy_paste_aug(images, bboxes, category_ids)
        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]

        # Forward prop.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

## validation

In [11]:
def validation(val_loader, model, criterion):
    model.eval()

    num_batches = len(val_loader)
    losses = AverageMeter()
    with torch.no_grad():
        for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
            images = images.to(device)
            bboxes = [b.to(device) for b in bboxes]
            category_ids = [l.to(device) for l in category_ids]

            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

            losses.update(loss, images.size(0))

    val_loss = losses.avg
    return val_loss

## checkpoint

In [12]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [13]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SSD300(n_classes=args.n_classes,
                                    base=args.base_model,
                                    ch_option=args.ch_option)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [14]:
workers = 4
train_dataset = dataset.KaistPDDataset()
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [15]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [ ]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10


# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                       model=model,
                       criterion=criterion,
                       optimizer=optimizer)
    run['train/loss'].log(train_loss)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-08-08 10:56:36.035983


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 112.908 (112.908)	Data Time 109.604 (109.604)	Loss 13.3130 (13.3130)	
[68/273]	Batch Time 32.480 (10.304)	Data Time 32.123 (9.913)	Loss 3.2081 (4.3582)	
[136/273]	Batch Time 33.649 (9.554)	Data Time 33.294 (9.184)	Loss 3.0995 (3.7827)	
[204/273]	Batch Time 30.956 (9.271)	Data Time 30.605 (8.907)	Loss 2.6757 (3.4958)	
[272/273]	Batch Time 20.510 (9.113)	Data Time 18.260 (8.745)	Loss 2.2592 (3.2757)	
# ====== Epoch 2 ====== # 2021-08-08 11:38:03.965595
[0/273]	Batch Time 3.933 (3.933)	Data Time 3.408 (3.408)	Loss 2.4525 (2.4525)	
[68/273]	Batch Time 2.034 (0.980)	Data Time 1.518 (0.450)	Loss 2.3469 (2.3255)	
[136/273]	Batch Time 0.577 (0.953)	Data Time 0.001 (0.428)	Loss 1.9028 (2.2321)	
[204/273]	Batch Time 0.532 (0.942)	Data Time 0.003 (0.414)	Loss 1.8521 (2.1739)	
[272/273]	Batch Time 0.290 (0.928)	Data Time 0.000 (0.404)	Loss 1.9107 (2.1205)	
# ====== Epoch 3 ====== # 2021-08-08 11:42:17.410334
[0/273]	Batch Time 4.766 (4.766)	Data Time 4.243 (4.243)	Loss 1.9286 (1

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.30s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 35.09%
Recall: 0.773851590106007
# ====== Epoch 11 ====== # 2021-08-08 12:37:25.974318


...

[0/273]	Batch Time 3.987 (3.987)	Data Time 3.429 (3.429)	Loss 1.0289 (1.0289)	
[68/273]	Batch Time 1.375 (0.967)	Data Time 0.833 (0.443)	Loss 1.1746 (1.1857)	
[136/273]	Batch Time 1.412 (0.954)	Data Time 0.905 (0.431)	Loss 1.0539 (1.1934)	
[204/273]	Batch Time 1.347 (0.942)	Data Time 0.822 (0.419)	Loss 1.1063 (1.2090)	
[272/273]	Batch Time 0.287 (0.930)	Data Time 0.000 (0.411)	Loss 1.0959 (1.2147)	
# ====== Epoch 122 ====== # 2021-08-08 20:29:57.001062
[0/273]	Batch Time 3.715 (3.715)	Data Time 3.051 (3.051)	Loss 1.4144 (1.4144)	
[68/273]	Batch Time 0.491 (0.957)	Data Time 0.001 (0.442)	Loss 1.2448 (1.2368)	
[136/273]	Batch Time 1.251 (0.926)	Data Time 0.758 (0.411)	Loss 1.1855 (1.2302)	
[204/273]	Batch Time 0.548 (0.908)	Data Time 0.005 (0.390)	Loss 0.9841 (1.2186)	
[272/273]	Batch Time 0.292 (0.904)	Data Time 0.000 (0.386)	Loss 1.3065 (1.2193)	
# ====== Epoch 123 ====== # 2021-08-08 20:34:03.978508
[0/273]	Batch Time 3.850 (3.850)	Data Time 3.267 (3.267)	Loss 1.4924 (1.4924)	
[68/273

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 28.27%
Recall: 0.8294573643410853
# ====== Epoch 131 ====== # 2021-08-08 21:08:04.896421


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 3.389 (3.389)	Data Time 2.882 (2.882)	Loss 1.3342 (1.3342)	
[68/273]	Batch Time 1.198 (0.931)	Data Time 0.625 (0.416)	Loss 1.2383 (1.1976)	
[136/273]	Batch Time 2.040 (0.923)	Data Time 1.536 (0.406)	Loss 1.2065 (1.1901)	
[204/273]	Batch Time 1.728 (0.924)	Data Time 1.196 (0.408)	Loss 1.1868 (1.1900)	
[272/273]	Batch Time 0.288 (0.908)	Data Time 0.000 (0.395)	Loss 0.9761 (1.1957)	
# ====== Epoch 132 ====== # 2021-08-08 21:12:12.841051
DECAYING learning rate.
 The new LR is 0.000050

[0/273]	Batch Time 4.328 (4.328)	Data Time 3.781 (3.781)	Loss 1.1113 (1.1113)	
[68/273]	Batch Time 2.049 (0.953)	Data Time 1.493 (0.427)	Loss 1.1203 (1.1529)	
[136/273]	Batch Time 0.587 (0.931)	Data Time 0.002 (0.407)	Loss 1.0876 (1.1366)	
[204/273]	Batch Time 1.228 (0.924)	Data Time 0.727 (0.401)	Loss 1.2117 (1.1323)	
[272/273]	Batch Time 0.292 (0.909)	Data Time 0.000 (0.390)	Loss 1.1984 (1.1263)	
# ====== Epoch 133 ====== # 2021-08-08 21:16:21.121323
[0/273]	Batch Time 3.709 (3.709)	Data

In [ ]:
# import eval

# checkpoint = torch.load(checkpoint_path)
# save_epoch = checkpoint['epoch']
# print(f'Loaded checkpoint from epoch {save_epoch}\n')
# model = checkpoint['model']
# eval.evaluate(model,
#               min_score=0.2,
#               max_overlap=0.5,
#               top_k=200)